# AdventureWorks - Sales Analysis

This sales analysis uses data from the AdventureWorks2025 database, accessed through SQL Server. Python is employed to retrieve the data and generate visualizations that support a clear and insightful analysis. While the majority of data processing and transformation is performed in SQL, additional transformations are applied in Python when needed to enhance the visualizations.

The analysis, presented in this *Jupyter Notebook*, addresses a series of predefined business questions, alongside one in-depth, supplementary investigation designed to provide deeper insights into AdventureWorks' product portfolio.

## Code setup

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
from matplotlib.patches import Patch
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

user = "SA"
password = quote_plus("Password_1234")
server = 'localhost:1433'
database = 'AdventureWorks2025'
driver = quote_plus("ODBC Driver 18 for SQL server")


connection_string = (
    f"mssql+pyodbc://{user}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

try:
    with engine.connect():
        print("Anslutning till SQL Server lyckades")
except Exception as e:
    print("Kunde inte ansluta", e)

# Helper function to execute SQL queries
def query_df(sql: str):
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn)

## Analysis

### Products per category

This visualisation analyses how many active products exsist in each category. 

**Database tables:** *Production.Product.Category*, *Production.ProductSubcategory*, *Production.Product*

**Method:** Active products were identified by filtering out discontinued products. Product categories were linked to their corresponding subcategories and products using inner joins. The number of distinct active products was then aggregated per category using a count (distinct) operation. The resulting data was visualised as a bar chart to compare product counts across categories.


In [ ]:
query_vis1 = """ 
SELECT
    pc.Name AS CategoryName,
    COUNT(DISTINCT p.ProductID) AS ProductCount
FROM Production.ProductCategory pc
INNER JOIN Production.ProductSubcategory psc 
    ON pc.ProductCategoryID = psc.ProductCategoryID
INNER JOIN Production.Product p 
    ON psc.ProductSubcategoryID = p.ProductSubcategoryID
WHERE p.DiscontinuedDate IS NULL
GROUP BY pc.Name
ORDER BY ProductCount DESC
"""

df_vis1 = query_df(query_vis1)

fig, ax = plt.subplots(figsize=(10,6))
bars = ax.bar(df_vis1['CategoryName'], df_vis1['ProductCount'], color='lightgreen', alpha=0.8)

for bar in bars: 
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f'{int(height)}',
        ha='center',
        va='bottom'
    )

ax.set_xlabel('Product Category', fontsize=12)
ax.set_ylabel('Number of products', fontsize=12)
ax.set_title('Number of products per category', fontsize=14)
ax.grid(axis='y', linestyle='--', alpha=0.4)

plt.tight_layout()
plt.show()

#### Insights - Products per categories

- Components is the largest product category, comprising 134 products and accounting for nearly half of the total product assortment. This is followed by Bikes, which includes 97 distinct products.

- Clothing and Accessories represent significantly smaller product categories in comparison.

- These findings indicate that the company primarily focuses on Bikes and Components rather than Clothing and Accessories.

### Revenue per category

This visualisation analyses which product categories generate the most revenue.

**Database tables:** *Production.ProductCategory*, *Production.ProductSubcategory*, *Production.Product*, *Sales.SalesOrderDetail*

**Method:** Sales order data was joined with product, subcategory, and category tables to associate each sale with its corresponding product category. Revenue was calculated by aggregating the line totals for each category using a sum operation. The aggregated revenue per category was then visualised using a horizontal bar chart to enable comparison across categories.

In [ ]:
query_vis2 = """ 
SELECT
    pc.Name AS CategoryName,
    CAST(ROUND(SUM(sod.LineTotal), 0) AS BIGINT) AS CategoryRevenue
FROM Sales.SalesOrderDetail sod
LEFT JOIN Production.Product p ON sod.ProductID = p.ProductID
LEFT JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
LEFT JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
GROUP BY pc.Name
ORDER BY CategoryRevenue DESC
"""

df_vis2 = query_df(query_vis2)

fig, ax = plt.subplots(figsize=(10,6))
bars = ax.barh(df_vis2['CategoryName'], df_vis2['CategoryRevenue'], color='lightgreen', alpha=0.8)

for bar in bars: 
    value = bar.get_width()
    ax.text(
        value * 1.01,
        bar.get_y() + bar.get_height() / 2,
        f'{value:,.0f}',
        va='center',
        ha='left'
    )

ax.set_xlabel('Revenue per Category', fontsize=12)
ax.set_ylabel('Category', fontsize=12)
ax.set_title('Total Revenue per Category', fontsize=14)
ax.grid(axis='x', linestyle='--', alpha=0.4)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

#### Insights - Revenue per category

- The Bike category generates by far the highest revenue, exceeding 94 million. It is also the second-largest category in terms of number of products.

- Although Components contains the largest number of products, it is not the primary driver of revenue.

- Clothing and Accessories generate the lowest revenue, at approximately 2.1 million and 1.3 million respectively. This aligns with their position as the company’s smallest product segments.

### Revenue Trend Overtime

This visualisation analyses how monthly revenue has developed over time.

**Database tables:** *Sales.SalesOrderHeader*

**Method:** Order dates were transformed to a monthly level to enable time-based aggregation. Total sales were calculated by summing the order subtotals for each month. The aggregated monthly revenue was then visualised using a line chart to illustrate revenue trends over time.

In [ ]:
query_vis3 = """ 
SELECT
    DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1) AS OrderMonth,
    SUM(SubTotal) AS TotalSales
FROM Sales.SalesOrderHeader
GROUP BY DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1)
ORDER BY OrderMonth ASC;
"""

df_vis3 = query_df(query_vis3)

fix, ax = plt.subplots(figsize=(12, 6))
ax.plot(df_vis3["OrderMonth"], df_vis3["TotalSales"], marker='o')

ax.set_xlabel("Month")
ax.set_ylabel("Total Sales")
ax.set_title("Monthly Sales Trend Overtime")
ax.grid(axis='y', linestyle='--', alpha=0.4)

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

#### Insights - Revenue Trend Overtime

- The chart illustrates the development of monthly sales over the period from May 2022 to June 2025.

- Overall, sales show a gradual upward trend across the period, with clear seasonal fluctuations. Revenue tends to increase during early summer, particularly between May and July, most notably in 2023 and 2024.

- After the summer period, sales increase again during August and September 2024, followed by a decline in October and a subsequent rise in November and December.

- The year 2025 shows a strong start, with the highest sales levels in the observed period occurring in February and March. This is followed by a sharp decline in subsequent months.

### Total sales and total orders per year

A grouped bar chart displays total sales and total orders using separate y-axes to account for differences in scale.

**Database tables:** *Sales.SalesOrderHeader*

**Method:** Order data was aggregated at a yearly level by extracting the year from the order date. Total sales were calculated by summing order subtotals, while the total number of orders was calculated by counting sales orders for each year. The results were visualised using a grouped bar chart with dual y-axes to allow comparison between total sales and order volume despite differences in scale.

In [ ]:
query_vis4 = """ 
SELECT
    YEAR(orderDate) AS OrderYear,
    CAST(ROUND(SUM(SubTotal), 0) AS BIGINT) AS TotalYearSales,
    COUNT(SalesOrderID) AS TotalOrders
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate)
ORDER BY OrderYear ASC
"""

df_vis4 = query_df(query_vis4)

x = np.arange(len(df_vis4['OrderYear']))
width = 0.35

fig, ax1 = plt.subplots(figsize=(10, 6))

# Left axis: Total Sales
bars_sales = ax1.bar(
    x - width/2,
    df_vis4['TotalYearSales'],
    width,
    label='Total Sales',
    color='steelblue',
    alpha=0.8
)

# Right axis: Total Orders
ax2 = ax1.twinx()

bars_orders = ax2.bar(
    x + width/2,
    df_vis4['TotalOrders'],
    width,
    label='Total Orders',
    color='lightgreen',
    alpha=0.8
)

ax1.set_xticks(x)
ax1.set_xticklabels(df_vis4['OrderYear'])
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Total Sales', fontsize=12)
ax1.grid(axis='y', linestyle='--', alpha=0.4)
ax2.set_ylabel('Total Orders', fontsize=12)

ax1.yaxis.set_major_formatter(
    mtick.FuncFormatter(lambda x, pos: f'{x/1_000_000:.1f}M')
)

fig.suptitle('Total Sales and Total Orders per Year', fontsize=14)

legend_handles = [
    Patch(facecolor='steelblue', label='Total Sales'),
    Patch(facecolor='lightgreen', label='Total Orders')
]
ax1.legend(handles=legend_handles, loc='upper left')

for bar in bars_sales:
    height = bar.get_height()
    ax1.text(
        bar.get_x() + bar.get_width()/2,
        height * 1.01, 
        f'{height:,.0f}',
        ha='center',
        va='bottom',
        fontsize=10
    )

for bar in bars_orders:
    height = bar.get_height()
    ax1.text(
        bar.get_x() + bar.get_width()/2,
        0, 
        f'{int(height):,}',
        ha='center',
        va='bottom',
        fontsize=10,
    )

plt.tight_layout()
plt.show()


#### Insights - Total sales and total orders per year

- The chart illustrates total sales and total orders per year from 2022 to 2025. It should be noted that the figures for 2025 only include data up to May and therefore do not represent a full year.

- Total sales and total orders appear to be positively correlated, as an increase in the number of orders is associated with higher total sales.

- The strongest performance is observed in 2024 - however, if the current trend continues, 2025 has the potential to exceed the total sales recorded in 2024.

### Top 10 products

This visualisation analyses the top ten products generating the highest sales. The figures are aggregated over the time period of May 2022 to June 2025. 

**Database tables:** *Production.Product*, *Sales.SalesOrderDetail*

**Method:** Sales order detail data was joined with product information to associate each sale with its corresponding product. Total sales were aggregated for each product, and the ten products with the highest sales values were selected. The results were visualised using a horizontal bar chart to enable comparison between the top-performing products.

In [ ]:
query_vis5 = """ 
SELECT TOP 10
    p.ProductID,
    p.Name AS ProductName,
    CAST(ROUND(SUM(sod.LineTotal), 0) AS BIGINT) AS TotalProductSales
FROM Sales.SalesOrderDetail sod
INNER JOIN Production.Product p ON sod.ProductID = p.ProductID
GROUP BY p.ProductID, p.Name
ORDER BY TotalProductSales DESC;
"""

df_vis5 = query_df(query_vis5)

fig, ax = plt.subplots(figsize=(10,6))
bars = ax.barh(df_vis5['ProductName'], df_vis5['TotalProductSales'], color='lightgreen', alpha=0.8)

for bar in bars: 
    value = bar.get_width()
    ax.text(
        value * 1.01,
        bar.get_y() + bar.get_height() / 2,
        f'{value:,.0f}',
        va='center',
        ha='left'
    )

ax.set_xlabel('Total Product Sales', fontsize=12)
ax.set_ylabel('Product', fontsize=12)
ax.set_title('Top 10 products by Sales', fontsize=14)
ax.grid(axis='x', linestyle='--', alpha=0.4)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

#### Insights - Top 10 products

- The chart displays the top ten products by total sales value and is dominated by products from the Bike category. These consist primarily of Mountain and Road bikes offered in different sizes and colours.

- The highest-selling product by sales value is the Mountain-200 Black, 38, which has generated approximately 4.4 million in sales.

### Total sales and customer per region

This visualisation analyses how sales differ between regions and how many unique customers there are in each region. The figures are aggregated over the time period of May 2022 to June 2025. 

**Database tables:** *Sales.SalesTerritory*, *Sales.SalesOrderHeader*, *Sales.Customer*

**Method:** Sales territory data was combined with sales order and customer information to associate each order with a specific region. Total sales were aggregated per region, and the number of unique customers was calculated to measure customer reach. The aggregated results were visualised using a grouped bar chart with dual y-axes to allow comparison between total sales and customer counts across regions.

In [ ]:
query_vis6 = """ 
SELECT
    CONCAT(st.Name, ', ', st.CountryRegionCode) AS RegionNameCode,
    CAST(ROUND(SUM(soh.SubTotal), 0) AS BIGINT) AS TotalRegionSales,
    COUNT(DISTINCT c.CustomerID) AS UniqueCustomers
FROM Sales.SalesTerritory st
INNER JOIN Sales.SalesOrderHeader soh 
    ON st.TerritoryID = soh.TerritoryID
INNER JOIN Sales.Customer c 
    ON st.TerritoryID = c.TerritoryID
GROUP BY st.Name, st.CountryRegionCode
ORDER BY TotalRegionSales DESC;
"""

df_vis6 = query_df(query_vis6)

x = np.arange(len(df_vis6['RegionNameCode']))
width = 0.35

fig, ax1 = plt.subplots(figsize=(10, 6))

# Left axis: Total Sales
bars_sales = ax1.bar(
    x - width/2,
    df_vis6['TotalRegionSales'],
    width,
    label='Total Sales',
    color='steelblue',
    alpha=0.8
)

# Right axis: Total Customers
ax2 = ax1.twinx()

bars_orders = ax2.bar(
    x + width/2,
    df_vis6['UniqueCustomers'],
    width,
    label='Total Customers',
    color='lightgreen',
    alpha=0.8
)

ax1.set_xticks(x)
ax1.set_xticklabels(df_vis6['RegionNameCode'])
ax1.tick_params(axis='x', labelrotation=60)
ax1.set_xlabel('Region', fontsize=12)
ax1.set_ylabel('Total Sales', fontsize=12)
ax1.grid(axis='y', linestyle='--', alpha=0.4)
ax2.set_ylabel('Unique Customers', fontsize=12)

ax1.yaxis.set_major_formatter(
    mtick.FuncFormatter(lambda x, pos: f'{x/1_000_000:.1f}M')
)

fig.suptitle('Total Sales and Customer per Region', fontsize=14)

legend_handles = [
    Patch(facecolor='steelblue', label='Total Sales'),
    Patch(facecolor='lightgreen', label='Unique Customers')
]
ax1.legend(handles=legend_handles, loc='upper right')

plt.tight_layout()
plt.show()


#### Insights - Total Sales and Customers per Region

- The Southwest region in the US is the strongest in terms of both total sales and number of unique customers, holding the highest figures among all regions.

- The Northeast, Central, and Southeast regions in the US have the lowest sales values and the fewest unique customers.

- While a higher number of unique customers generally corresponds to higher total sales, the relationship does not appear strictly proportional. For example, Australia shows a relatively high number of unique customers but comparatively low total sales.

### AOV per region and customer type

This visualisation analyses the average order value (AOV) across regions and customer types, comparing store and individual customers. The figures are aggregated over the timeperiod of May 2022 to June 2025. 

**Database tables:** *Sales.SalesTerritory*, *Sales.SalesOrderHeader*, *Sales.Customer*, *Sales.Store*

**Method:** Sales order data was combined with customer, store, and territory information to associate each order with a region and a customer type (store or individual). Average order value (AOV) was calculated per region and customer type by dividing total sales by the number of orders. The top ten regions based on overall AOV were selected and visualised using a grouped bar chart to compare AOV between store and individual customers across regions.

In [ ]:
query_vis7 = """ 
SELECT
    CONCAT(st.Name, ', ', st.CountryRegionCode) AS Region,
    CASE
        WHEN s.BusinessEntityID IS NULL THEN 'Individual'
        ELSE 'Store'
    END AS CustomerType,
    SUM(soh.SubTotal) / COUNT(DISTINCT soh.SalesOrderID) AS AverageOrderValue
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.Customer c 
    ON soh.CustomerID = c.CustomerID
LEFT JOIN Sales.Store s
    ON c.StoreID = s.BusinessEntityID
INNER JOIN Sales.SalesTerritory st
    ON c.TerritoryID = st.TerritoryID
GROUP BY 
    st.Name, 
    st.CountryRegionCode,
    CASE
        WHEN s.BusinessEntityID IS NULL THEN 'Individual'
        ELSE 'Store'
    END
ORDER BY AverageOrderValue DESC;
"""

df_vis7 = query_df(query_vis7)

df_pivot = df_vis7.pivot(
    index='Region',
    columns='CustomerType',
    values='AverageOrderValue'
).fillna(0)

df_pivot["TotalAvg"] = df_pivot.mean(axis=1)
df_pivot = df_pivot.sort_values("TotalAvg", ascending=False)

df_pivot = df_pivot.head(10)

x = np.arange(len(df_pivot))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))

bars_store = ax.bar(
    x - width/2,
    df_pivot['Store'],
    width, 
    color='steelblue',
    label='Store'
)

bars_individual = ax.bar(
    x + width/2,
    df_pivot['Individual'],
    width,
    color='lightgreen',
    label='Individual'
)

ax.set_title("AOV per region and customer type", fontsize=14)
ax.set_xlabel("Region")
ax.set_ylabel("AOV")

ax.set_xticks(x)
ax.set_xticklabels(df_pivot.index, rotation=45, ha="right")

ax.legend()
ax.grid(axis="y", linestyle="--", alpha=0.4)

for bars in [bars_store, bars_individual]:
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            height,
            f'{height:,.0f}',
            ha='center',
            va='bottom',
            fontsize=8
            )
        
plt.tight_layout()
plt.show()

#### Insights - AOV per region and customer type

- Store customers have a significantly higher average order value (AOV) than individual customers in all regions.

- The gap between store and individual AOV is especially large in Southwest (US), France, and the United Kingdom, where store orders are more than 20 times larger than individual orders.

- Individual customer AOV is consistently low across all regions, with the highest in Australia and the lowest in Central, US.

- The highest AOV overall is observed for store customers in the Southwest, US, making this the dominant region and customer type combination.

- Regional differences may reflect factors such as local pricing, bulk ordering behavior, and customer segment composition.

## Product portfolio analysis

In this section, AdventureWorks’ product portfolio is analysed to evaluate product performance and identify opportunities for strategic improvement. The analysis focuses on sales volume, revenue, and profitability in order to distinguish high-performing products from underperforming ones.

To guide the analysis, a set of predefined business questions is addressed, including:

- Which products can be classified as “winners,” characterised by high sales and high profit margins?
- Which products are “losers,” defined by low sales or negative margins?
- Which products should potentially be discontinued, and which should receive increased strategic focus?
- Are there products with high sales quantities but relatively low revenue contributions?

### Winner/loser-products



**Database tables:** *Sales.SalesOrderDetail, Production.ProductSubcategory, Production.ProductCategory*

**Method:** Product performance was analysed by aggregating sales data in SQL to calculate total sales, gross margin, and margin percentage for each active product. Products are classified as *Winners* if they have both higher sales and higher margins than average. Products with negative margins or low sales are classified as *Losers*. The remaining products fall into the *Middle* category, indicating stable but suboptimal performance.

The results were visualised to show the distribution of product statuses across categories (*matrix*), as well as to highlight the top ten products with the highest gross margins and the bottom ten with the lowest gross margins (*bar chart*).

In [ ]:
query_vis8 = """ 
WITH ProductPerformance AS (
    SELECT
        sod.ProductID,
        p.Name AS ProductName,
        pc.Name AS Category,
        psc.Name AS Subcategory,
        SUM(sod.OrderQty) AS TotalQuantity,
        SUM(sod.LineTotal) AS TotalSales,
        SUM(sod.OrderQty * p.StandardCost) AS TotalCost,
        SUM(sod.LineTotal) - SUM(sod.OrderQty * p.StandardCost) AS GrossMargin,
        (SUM(sod.LineTotal) - SUM(sod.OrderQty * p.StandardCost))
            / NULLIF(SUM(sod.LineTotal), 0) AS MarginPercent
    FROM Sales.SalesOrderDetail sod
    JOIN Production.Product p
        ON sod.ProductID = p.ProductID
    LEFT JOIN Production.ProductSubcategory psc
        ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    LEFT JOIN Production.ProductCategory pc
        ON psc.ProductCategoryID = pc.ProductCategoryID
    WHERE p.DiscontinuedDate IS NULL
    GROUP BY
        sod.ProductID,
        p.Name,
        pc.Name,
        psc.Name
)

SELECT 
    ProductID,
    ProductName,
    Category,
    Subcategory,
    TotalQuantity,
    TotalSales,
    GrossMargin,
    MarginPercent,

    CASE
        WHEN TotalSales >= (SELECT AVG(TotalSales) FROM ProductPerformance)
        AND MarginPercent >= (SELECT AVG(MarginPercent) FROM ProductPerformance)
            THEN 'Winner'
        
        WHEN GrossMargin < 0
        OR TotalSales < (SELECT AVG(TotalSales) FROM ProductPerformance)
            THEN 'Loser'
        
        ELSE 'Middle'
    END AS ProductStatus

FROM ProductPerformance
ORDER BY TotalSales DESC;
"""

df_vis8 = query_df(query_vis8)

df_pivot = pd.pivot_table(
    df_vis8,
    index='Category',
    columns='ProductStatus',
    values='ProductID',
    aggfunc='count',
    fill_value=0
    )

# Matrix - product status per category
status_colors = {
    'Winner': 'lightgreen',
    'Middle': 'khaki',
    'Loser': 'salmon'
}

default_color = 'lightgrey'

data = df_pivot.values
categories = df_pivot.index.tolist()
statuses = df_pivot.columns.tolist()

fig, ax = plt.subplots(figsize=(len(statuses)*2, len(categories)*1.2))

for i, cat in enumerate(categories):
    for j, status in enumerate(statuses):
        count = data[i, j]
        color = status_colors.get(status, default_color)
        
        rect = plt.Rectangle((j, i), 1, 1, color=color, ec='black')
        ax.add_patch(rect)
        
        ax.text(j + 0.5, i + 0.5, str(count),
                ha='center', va='center', color='black', fontsize=10)

# --- Steg 5: Axlar och etiketter ---
ax.set_xticks(np.arange(len(statuses)) + 0.5)
ax.set_xticklabels(statuses)
ax.set_yticks(np.arange(len(categories)) + 0.5)
ax.set_yticklabels(categories)
ax.set_xlim(0, len(statuses))
ax.set_ylim(0, len(categories))
ax.set_xlabel('Product status')
ax.set_ylabel('Category')
ax.set_title('Product status per category')

ax.invert_yaxis()
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

# Bar chart - Top/bottom 10 
top_winners = df_vis8.sort_values('GrossMargin', ascending=False).head(10)
bottom_losers = df_vis8.sort_values('GrossMargin', ascending=True).head(10)

df_plot = pd.concat([top_winners, bottom_losers], ignore_index=True)

df_plot['Color'] = df_plot['ProductStatus'].map({
    'Winner': 'lightgreen',
    'Loser': 'salmon'
}).fillna('lightgrey')

plt.figure(figsize=(12, 6))

plt.bar(
    df_plot['ProductName'],
    df_plot['GrossMargin'],
    color=df_plot['Color'],
)

for idx, row in df_plot.iterrows():
    plt.text(
        idx, row['GrossMargin'] + 500,  
        f"{row['GrossMargin']:.0f}",
        ha='center', va='bottom', fontsize=9
    )

plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylabel('Gross Margin')
plt.title('Products - Top 10 winners and bottom 10 losers')
plt.tight_layout()
plt.show()

### Exit/Invest-analysis

**Database tables:** *Sales.SalesOrderDetail, Production.Product, Production.ProductSubcategory, Production.ProductCategory*

**Method:** An exit–invest assessment was conducted by aggregating product-level sales and profitability metrics in SQL. Products were classified into Invest, Keep/Opt, or Exit categories based on gross margin, total sales, and margin percentage relative to overall product averages.

The visualisations shows the distribution of recommendations across product categories and to highlight the top ten products recommended for investment and the bottom ten products recommended for exit based on gross margin performance.

In [ ]:
query_vis9 = """ 
WITH ProductPerformance AS (
    SELECT
        sod.ProductID,
        p.Name AS ProductName,
        pc.Name AS Category,
        psc.Name AS Subcategory,
        SUM(sod.OrderQty) AS TotalQuantity,
        SUM(sod.LineTotal) AS TotalSales,
        SUM(sod.OrderQty * p.StandardCost) AS TotalCost,
        SUM(sod.LineTotal) - SUM(sod.OrderQty * p.StandardCost) AS GrossMargin,
        (SUM(sod.LineTotal) - SUM(sod.OrderQty * p.StandardCost))
            / NULLIF(SUM(sod.LineTotal), 0) AS MarginPercent
    FROM Sales.SalesOrderDetail sod
    JOIN Production.Product p
        ON sod.ProductID = p.ProductID
    LEFT JOIN Production.ProductSubcategory psc
        ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    LEFT JOIN Production.ProductCategory pc
        ON psc.ProductCategoryID = pc.ProductCategoryID
    WHERE p.DiscontinuedDate IS NULL
    GROUP BY
        sod.ProductID,
        p.Name,
        pc.Name,
        psc.Name
)
SELECT 
    ProductID,
    ProductName,
    Category,
    Subcategory,
    TotalQuantity,
    TotalSales,
    GrossMargin,
    MarginPercent,

    CASE
        WHEN GrossMargin < 0
            THEN 'Exit'
        
        WHEN TotalSales < (SELECT AVG(TotalSales) FROM ProductPerformance)
            AND MarginPercent < (SELECT AVG(MarginPercent) FROM ProductPerformance)
            THEN 'Exit'

        WHEN TotalSales >= (SELECT AVG(TotalSales) FROM ProductPerformance)
            AND MarginPercent >= (SELECT AVG(MarginPercent) FROM ProductPerformance)
            THEN 'Invest'
        
        ELSE 'Keep/Opt'
    END AS Recommendation

FROM ProductPerformance
ORDER BY Recommendation, TotalSales DESC;
   
"""

df_vis9 = query_df(query_vis9)

df_pivot = pd.pivot_table(
    df_vis9,
    index='Category',
    columns='Recommendation',
    values='ProductID', 
    aggfunc='count',
    fill_value=0
)

# Bar chart: Products per category and recommendation
desired_order = ['Invest', 'Keep/Opt', 'Exit']
df_pivot = df_pivot[desired_order]

categories = df_pivot.index.tolist()
recommendations = df_pivot.columns.tolist()

x = np.arange(len(categories)) 
width = 0.25  

colors = {
    'Invest': 'lightgreen',
    'Keep/Opt': 'khaki',
    'Exit': 'salmon'
}

fig, ax = plt.subplots(figsize=(10, 6))

for i, rec in enumerate(recommendations):
    ax.bar(
        x + i*width,
        df_pivot[rec].values,
        width,
        label=rec,
        color=colors.get(rec, 'grey')
    )

ax.set_xticks(x + width*(len(recommendations)-1)/2)
ax.set_xticklabels(categories, rotation=45, ha='right')
ax.set_ylabel('Product count')
ax.set_xlabel('Category')
ax.set_title('Products per category and recommendation')
ax.legend(title='Recommendation')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Bar chart: Top 10 invest/bottom 10 exit
top_winners = df_vis9.sort_values('GrossMargin', ascending=False).head(10)
bottom_losers = df_vis9.sort_values('GrossMargin', ascending=True).head(10)

df_plot = pd.concat([top_winners, bottom_losers], ignore_index=True)

df_plot['Color'] = df_plot['Recommendation'].map({
    'Invest': 'lightgreen',
    'Exit': 'salmon'
}).fillna('lightgrey')

plt.figure(figsize=(12, 6))

plt.bar(
    df_plot['ProductName'],
    df_plot['GrossMargin'],
    color=df_plot['Color'],
)

for idx, row in df_plot.iterrows():
    plt.text(
        idx, row['GrossMargin'] + 500, 
        f"{row['GrossMargin']:.0f}",
        ha='center', va='bottom', fontsize=9
    )

plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylabel('Gross Margin')
plt.title('Products - Top 10 invest and bottom 10 exit')
plt.tight_layout()
plt.show()

### High quantity, low revenue

**Database tables:** *Sales.SalesOrderDetail, Production.Product, Production.ProductSubcategory, Production.ProductCategory*

**Method:** The analysis identifies products that are sold in high volumes but generate low revenue per unit. Products were analysed by aggregating total quantity sold and average revenue per unit at the product level using SQL. Products with above-average sales volumes and below-average revenue per unit were identified to highlight items with high turnover but relatively low revenue contribution.

The top ten products meeting these criteria has been visualised in a horisontal bar chart showing average revenue per unit as well as number of sold units. 

In [ ]:
query_vis10 = """ 
WITH ProductPerformance AS (
    SELECT
        sod.ProductID,
        p.Name AS ProductName,
        pc.Name AS Category,
        psc.Name AS Subcategory,
        SUM(sod.OrderQty) AS TotalQuantity,
        SUM(sod.LineTotal) AS TotalSales,
        SUM(sod.OrderQty * p.StandardCost) AS TotalCost,
        SUM(sod.LineTotal) - SUM(sod.OrderQty * p.StandardCost) AS GrossMargin,
        (SUM(sod.LineTotal) - SUM(sod.OrderQty * p.StandardCost))
            / NULLIF(SUM(sod.LineTotal), 0) AS MarginPercent,
        (SUM(sod.LineTotal) / NULLIF(SUM(sod.OrderQty), 0)) AS AvgRevenuePerUnit
    FROM Sales.SalesOrderDetail sod
    JOIN Production.Product p
        ON sod.ProductID = p.ProductID
    LEFT JOIN Production.ProductSubcategory psc
        ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    LEFT JOIN Production.ProductCategory pc
        ON psc.ProductCategoryID = pc.ProductCategoryID
    WHERE p.DiscontinuedDate IS NULL
    GROUP BY
        sod.ProductID,
        p.Name,
        pc.Name,
        psc.Name
)

SELECT
    ProductID,
    ProductName,
    Category,
    Subcategory,
    TotalQuantity,
    TotalSales,
    AvgRevenuePerUnit,

    CASE
        WHEN TotalQuantity >= (SELECT AVG(TotalQuantity) FROM ProductPerformance)
         AND AvgRevenuePerUnit <= (SELECT AVG(AvgRevenuePerUnit) FROM ProductPerformance)
            THEN 'High qty / Low rev'
        ELSE 'Normal'
    END AS QuantityRevenueFlag

FROM ProductPerformance
WHERE
    TotalQuantity >= (SELECT AVG(TotalQuantity) FROM ProductPerformance)
    AND AvgRevenuePerUnit <= (SELECT AVG(AvgRevenuePerUnit) FROM ProductPerformance)
ORDER BY TotalQuantity DESC;
   
"""

df_vis10 = query_df(query_vis10)

df_top10 = df_vis10.head(10).copy()

colors = {
    'High qty / Low rev': 'khaki', 
    'Normal': 'lightgrey'             
}
df_top10['Color'] = df_top10['QuantityRevenueFlag'].map(colors)

fig, ax = plt.subplots(figsize=(10, 6))

y_pos = np.arange(len(df_top10))

ax.barh(
    y_pos,
    df_top10['TotalQuantity'],
    color=df_top10['Color'],
)

ax.set_yticks(y_pos)
ax.set_yticklabels(df_top10['ProductName'])
ax.invert_yaxis() 
ax.set_xlabel('Total Quantity Sold')
ax.set_title('Top 10 products: high volume sold, low revenue per item')

for i, val in enumerate(df_top10['AvgRevenuePerUnit']):
    ax.text(
        val + val*0.01, 
        i,
        f"${val:.2f} ({df_top10.iloc[i]['TotalQuantity']} units)",
        va='center',
        ha='left',
        fontsize=9
    )

plt.tight_layout()
plt.show()



### Insights - Product portfolio

- The product mix indicates a strategic focus on Bikes and Components as the two largest categories, while Clothing and Accessories appear to function as supplementary segments. Bikes generate by far the highest revenue, followed by Components, whereas Clothing and Accessories generate the lowest revenue, consistent with their smaller assortments.

- All identified “Winner” products (high sales and high margins) belong to the Bike category, which also dominates total revenue and the top ten products by sales value.

- “Loser” products are more numerous and distributed across all categories, with the largest concentration in Components. This reflects Components’ large assortment size and comparatively lower revenue contribution relative to Bikes.

- The exit–invest analysis shows that recommended investment opportunities are concentrated in Bikes. However, this category also contains a substantial number of products recommended for exit, alongside Components, which holds the highest number of exit-classified products.

- Products with high sales volumes but low revenue per unit are exclusively found in Clothing and Accessories. This aligns with these categories’ overall classification as “Loser” products, characterised by low sales value and low margins.